Test is a test notebook where I develop the GOR3 algorithm.

Things to do:

- The first thing we should do is import the data and clean the data to make it ready for use. 
- Understand the GOR 3 Algorithm
- Understand how we should deal with edges 
- Understand what we need to do with invalid entries and how we deal with it in algorithm
- Understand how we should make it optimised leave-one-out. (don't recount everything with every new protein sequence).
- How to summarise the results of the GOR3 (per data set)
- How to summarise the results per protein family.


In [293]:
import numpy as np
import pandas as pd

In [202]:
# Algorithm to evaluate a protein. 
# Create a random protein sequence. Where the amino acids are code between 0,20
Proteins = np.random.randint(3,size=100)
SeqLen = len(Proteins)

In [203]:
# Add some invalid entries and pad the sides with -1's which represent the edges.
Proteins[[15,74,87]] = -1 
#Proteins = np.concatenate((-np.ones(8),Proteins,-np.ones(8))).astype(int)

In [188]:
# Create for now a dummy frequency tensor:
freq = np.ones(shape=(3,17,3,3),dtype=float)
freq[0,8,0,0] = 7
freq[0,8,1,1] = 10
freq[0,8,2,2] = 13
freq[1,8,0,0] = 70
freq[1,8,1,1] = 100
freq[1,8,2,2] = 130


CountSSperR = np.ones(shape=(3,20))
CountSS = np.sum(CountSSperR,-1)
otherSS = np.array([[1,2],[0,2],[0,1]]) # Have the index of the other Secundary Structure such that we can sum of the other SS
logDiff = np.log(CountSS/np.sum(CountSS[otherSS],axis=-1))
logDiffRj = np.log(CountSSperR/np.sum(CountSSperR[otherSS],axis=-2)) # -2 since -1 are the amino acids

In [208]:
Proteins

array([ 1,  2,  0,  2,  2,  1,  0,  2,  2,  0,  1,  1,  0,  2,  1, -1,  2,
        0,  1,  0,  2,  2,  0,  1,  0,  2,  1,  2,  1,  1,  2,  0,  0,  1,
        1,  1,  2,  2,  1,  1,  2,  2,  1,  0,  1,  2,  1,  1,  1,  0,  1,
        2,  0,  2,  0,  2,  2,  2,  0,  1,  1,  0,  2,  1,  0,  2,  0,  1,
        1,  0,  2,  1,  2,  2, -1,  1,  2,  1,  1,  2,  2,  1,  1,  1,  2,
        1,  2, -1,  1,  1,  2,  1,  2,  0,  0,  0,  2,  1,  0,  0])

In [190]:
CountSS

array([20., 20., 20.])

In [169]:
valid = np.array([0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1]).astype(bool)
seq = np.arange(17)
vseq = seq[valid]
mCorrect = np.where(valid)[0]
freq[0,mCorrect,vseq,0]

IndexError: index 8 is out of bounds for axis 2 with size 3

In [196]:
testarray = np.array([4,8,7])
if 8 in testarray:
    print("Yes")

Yes


In [220]:
import time
startTime = time.time()
for i in range(500):
    out = predictSS(Proteins,freq,logDiff,logDiffRj)
Diff = time.time() - startTime
print(f"{np.round(Diff,2)} seconds")

45.34 seconds


In [376]:
def predictSS(protSequence, freq, logDiff, logDiffRj, SStrue, predictInvalidResidue=False):
    """
    Input:
        - protSequence : 1D-numpy array, Represents the proteins sequence, where the aminoacids are labeled as integers from 0 to 19
        - freq : 4D-numpy array shape = (3,17,20,20) representing the frequency count of the data set.
            - axis0 = Secundary structure
            - axis1 = relative position m from the window center positioned at j in the protein sequence
            - axis2 = amino Acid at the position m. = R_{j+m}
            - axis3 = amino Acid at the centre. = R_j
        - logDiff: 1D-numpy array of length 3. This is the logarithm of the relative SS structure count: log(f_s/f_{not_s})
        - logDiffRj: 2D-numpy array of shape (3,20). This is the logarithm of the relative SS structure count per amino accid: log(f_{s,R}/f_{not_s,R})
        - SStrue
        - dealWithInvalidResidue
    Output:
        - SS_result: 1D-numpy array with the predicted SS of the protein sequence
        - ISS: 2D-numpy array with the information per SS per amino accid in the protein sequence.
        
    """
    # Add 8 "-1"'s at the beginning and end of the protein sequence. 
    # This is done such that we can have running window of lenght 8 at each side at the edges of the protein sequences. 
    # Use -1 since we offcourse vieuw these as invalid entries.
    paddedSequence = np.concatenate((-np.ones(8),protSequence,-np.ones(8))).astype(int)
    SeqLen = len(protSequence)
    otherSS = np.array([[1,2],[0,2],[0,1]]) # Have the index of the other Secundary Structure such that we can sum of the other SS
    
    # Construct the output value ISS
    ISS = -np.ones(shape=(3,SeqLen)) # use -1 to later on detect invalid entries in the protein sequence.
    
    for j, r_j in enumerate(paddedSequence):
        if j-8 > SeqLen: # We have reached the end.
            break
        if r_j<0: # We skip over invalid entries. (These are the padded edges and invalid entries)
            continue
        # r_j is valid so set the ISS level equalt to zero. Unvalid r_j are kept -1
        ISS[:,j-8] = np.zeros(3)             
        for SS in range(3):
            for m, r_jm in enumerate(Proteins[j-8:j+9]): # m in [0,16],
                if r_jm<0:
                    continue
                # Compute the logSum
                ISS[SS,j-8] += np.log(freq[SS,m,r_jm,r_j]/np.sum(freq[otherSS[SS],m,r_jm,r_j]))
                if m==8: 
                    ISS[SS,j-8] += logDiff[SS]
                else:
                    ISS[SS,j-8] += logDiffRj[SS,r_j]

    SS_result = np.argmax(ISS,axis=0)
    
    if not predictInvalidResidue:
        SS_result[protSequence<0] = -1 # Let the invalid protein sequences stay flagged as invalid.
        return SS_result, ISS

    ###################################################
    ##### Deal with invalid amino acid entries ######
    ##################################################
    # This is not general yet !. 
    
    InvalidResidue = np.logical_and(protSequence<0 and trueSS>=0) # Make sure that invalid entry is not duo a skip. 
    for index in InvalidResidue:
        if index == 0: # The invalid is the first element
            indexToCopy = 1
            indexIsNotValid = np.any(Invalidindex==indexToCopy)
            while indexIsNotValid: # Keep searching until we are sure that the next index is also not an invalid one. 
                indexToCopy+=1
                nextIsValid = np.any(InvalidIndex==indexToCopy) and trueSS[indexToCopy]>=0
            SS_result[index] = SS_result[indexToCopy] # Set it equal to the first valid element. 
        else: # In all other cases we pick the one below
            SS_result[index] = SS_result[index-1]
        # The reason why we pick the one is clarified by the logic that we would have else used:

#       elif (index-1)==SeqLen # The invalid index is the last element
#           SS_result[index] = SS_result[index-1] # Set equal to the element before.
#       elif SS_result[index-1]==SS_result[index+1] # SS around in valid point is the same
#           SS_result[index] = SS_result[index-1]
#       else: # By default just pick the one below, this way we are sure the entry exist. 
#           SS_result[index]=SS_result[index-1] 

        # And so in all other cases it would have been pick the one below so. We can just simplify it with an easy else. 

    return SS_result, ISS # Return the found SS and the information values.

------
Left to do now is reading in the data file and computing the frequency tables.

In [263]:
def freqTableOfProtein(protSequence,SS):
    """
    Input:
        - protSequence : 1D-numpy array, Represents the proteins sequence, where the aminoacids are labeled as integers from 0 to 19
        - SS : 1D-numpy array holding the know secundary structure of the 
    Output:
    - freq : 4D-numpy array shape = (3,17,20,20) representing the frequency count of the protein sequence.
            - axis0 = Secundary structure
            - axis1 = relative position m from the window center positioned at j in the protein sequence
            - axis2 = amino Acid at the position m. = R_{j+m}
            - axis3 = amino Acid at the centre. = R_j
    - countSS : returns the total counts of each type of secundary structure in the sequence
    - coutSSperR: return the count of each type of secundary structure per amino acid.
    """
    freqTable = np.zeros(shape=(3,17,20,20))
    
    # Pad the sequence with 8 -1's before and after such that we can easily perform move a window around. 
    paddedSequence = np.concatenate((-np.ones(8),protSequence,-np.ones(8))).astype(int)
    paddedSS = np.concatenate((-np.ones(8),SS,-np.ones(8))).astype(int)
    
    seqLen = len(protSequence)
    
    #Fill freqTable
    for j, (r_j, S_j) in enumerate(zip(paddedSequence,paddedSS)):
        if j-8 > SeqLen: # We have reached the end.
            break
        if r_j<0: # We skip over invalid entries. (These are the padded edges and invalid entries)
            continue
        for m in np.arange(-8,9):#  m=[-8,...,8]
            r_jm = paddedSequence[j+m]
            if r_jm<0:
                continue
            freqTable[S_j,m+8,r_jm,r_j]+=1
            
    middleTable = freqTable[:,8] # take all the freq counts on m=0. 
    countSSperR = middleTable.diagonal(axis1=-1,axis2=-2) # Extract the diagonal
    countSS = np.sum(countSSperR,axis=-1)
    
    return freqTable, countSS, countSSperR

In [378]:
def leaveOneOutAnalysis(dataSet = "dssp", protFamily = None, predictInvalidResidue = False ):
    """
        Estimate the Q3 and CVV of the GOR method for a specific dataset using a crossvalidation of leave-one-out
    """
    totalFreqTable = np.zeros(shape=(3,17,20,20))
    totalCountSS = np.zeros(3)
    totalCountSSperR = np.zeros(shape=(3,20))
    otherSS = np.array([[1,2],[0,2],[0,1]]) # Have the index of the other Secundary Structure such that we can sum of the other SS
    
    # Select the correct proteinTuple based on dssp protFamily
    
    #First compute the full freqTable
    for proteinSequence, trueSS in proteinTuple:
        freqTableFromProt, countSSFromProt, countSSperRFromProt = freqTableOfProtein(proteinSequence, trueSS)
        totalFreqTable += freqTableFromProt
        totalCountSS += countSSFromProt
        totalCountSSperR += countSSperRFromProt
        
    
    numberProteins = len(AllProteins)
    
    Q3 = np.zeros(shape=(3,number))
    CVV = np.zeros(shape=(3,number))
    
    # Now do the leave-one-out:
    for i, (proteinSequence, trueSS) in enumerate(proteinTuple):
        # Remove the frequency counts of this protein.
        freqTableFromProt, countSSFromProt, countSSperRFromProt = freqTableOfProtein(proteinSequence, trueSS)
        tempFreqTable = totalFreqTable - freqTableFromProt
        tempCountSS = totalCountSS - countSSFromProt
        tempCountSSperR = totalCountSSperR - countSSperRFromProt
        
        templogDiff = np.log(tempCountSS/np.sum(tempCountSS[otherSS],axis=-1))
        templogDiffRj = np.log(tempCountSSperR/np.sum(tempCountSSperR[otherSS],axis=-2)) # -2 since -1 are the amino acids
        
        # Set Frequency that are equal to zero = 0.00001 inorder to avoid infinities.
        tempFreqTable[tempFreqTable == 0] = 0.00001
        
        # Perform the GOR3 prediction.
        SS_result, _ = predictSS(proteinSequence, tempFreqTable, templogDiff, templogDiffRj, trueSS, predictInvalidResidue)
        
        # Remove the dummy entrances, such that they not influence the Q3 and CVV calculations
        # predictInvalidResidue = True, then invalid residue have also tried to be predicted. 
        # If we then now also remove them with the following code.
        validEntries = SS_result>=0
        SS_result = SS_result[validEntries]
        trueSS = trueSS[validEntries]
            
        Q3[i] = proteinQ3(SS_result, trueSS)
        CVV[i] = proteinCVV(SS_result, trueSS)
    
    return Q3, CVV

In [228]:
def proteinQ3(predictedSS, trueSS):
    _, (numberHelix, numberSheet, numberCoil) = np.unique(trueSS, return_counts=True)
    TPHelix = np.sum(np.logical_and(predictedSS==0, trueSS==0)) #Only look where trueSS = Helix, see if they are then equal.
    TPSheet = np.sum(np.logical_and(predictedSS==1, trueSS==1))
    TPCoil =  np.sum(np.logical_and(predictedSS==2, trueSS==2))
    return np.array([TPHelix/numberHelix, TPSheet/numberSheet, TPCoil/numberCoil])

def computeCVV(TP,TN,FP,FN):
    numerator = TP*TN-FP*FN
    denominator = np.sqrt((TP+FP) * (TP+FN) * (TN+FP) * (TN+FN))
    return numerator/denominator

def proteinCVV(predictedSS, trueSS):
    #Helix
    TPHelix = np.sum(np.logical_and(predictedSS==0, trueSS==0))
    TNHelix = np.sum(np.logical_and(predictedSS!=0, trueSS!=0))
    FPHelix = np.sum(np.logical_and(predictedSS==0, trueSS!=0))
    FNHelix = np.sum(np.logical_and(predictedSS!=0, trueSS==0))
    CVVHelix = computeCVV(TPHelix,TNHelix,FPHelix,FNHelix)
    #Sheet
    TPSheet = np.sum(np.logical_and(predictedSS==1, trueSS==1))
    TNSheet = np.sum(np.logical_and(predictedSS!=1, trueSS!=1))
    FPSheet = np.sum(np.logical_and(predictedSS==1, trueSS!=1))
    FNSheet = np.sum(np.logical_and(predictedSS!=1, trueSS==1))
    CVVSheet = computeCVV(TPSheet,TNSheet,FPSheet,FNSheet)
    #Coil
    TPCoil = np.sum(np.logical_and(predictedSS==2, trueSS==2))
    TNCoil = np.sum(np.logical_and(predictedSS!=2, trueSS!=2))
    FPCoil = np.sum(np.logical_and(predictedSS==2, trueSS!=2))
    FNCoil = np.sum(np.logical_and(predictedSS!=2, trueSS==2))
    CVVCoil = computeCVV(TPCoil,TNCoil,FPCoil,FNCoil)
    return np.array([CVVHelix, CVVSheet, CVVCoil])

-------
So we have constructed all the relevant funtions. 
It is now time to extract the data from the input files and store them appropiately. 

In [379]:
# Define some dictionary maps.
aminoToNumber = {"ALA":0, "ARG":1, "ASN":2, "ASP":3, "CYS":4, "GLN":5, "GLU":6, "GLY":7, "HIS":8, "ILE":9, "LEU":10, "LYS":11, "MET":12, "PHE":13, "PRO":14, "SER":15, "THR":16, "TRP":17, "TYR":18, "VAL":19}
shortAminoToNumber = {"A":0, "R":1, "N":2, "D":3, "C":4, "Q":5, "E":6, "G":7, "H":8, "I":9, "L":10, "K":11, "M":12, "F":13, "P":14, "S":15, "T":16, "W":17, "Y":18, "V":19}
SStoNumber = {"Helix":0, "Beta":1, "Coil":2, "Other":2}
shortSStoNumber = {"H":0, "E":1, "C":2}
numberToStringSS = {0:"H", 1:"E", 2:"C"}

In [394]:
df = pd.read_csv("inputData/dssp_info.txt", sep="\t", header=None, names=["PDB_code", "PDB_chain_code", "PDB_seq_code", "residue_name", "SS"])
df

,PDB_code,PDB_chain_code,PDB_seq_code,residue_name,SS
0,1w0n,A,12,ILE,Coil
1,1w0n,A,13,THR,Beta
2,1w0n,A,14,LYS,Beta
3,1w0n,A,15,VAL,Beta
4,1w0n,A,16,GLU,Beta
...,...,...,...,...,...
71386,1ow4,A,114,VAL,Helix
71387,1ow4,A,115,ARG,Helix
71388,1ow4,A,116,ASN,Helix
71389,1ow4,A,117,SER,Coil


------
Convert residue names and secundary structures to numbers which represent the coordinates of the frequency tensors. Residue names that are not of the standard 20 will be converted to -1 and thus flagged as invalid residue entry. This way the running window of the GOR algorithm knows to ignore those entries.

We can then later also choise if we want to predict the SS of those entries.

In [395]:
df['residue_name'] = pd.to_numeric( df['residue_name'].map(aminoToNumber).fillna(-1), downcast='signed')# Fill non matching (aka the not standard amino acids) = -1
df["SS"] = pd.to_numeric( df["SS"].map(SStoNumber), downcast="signed")

Group by `PDB_code` and `PDB_chain_code` code, since there are instance of proteins in multiple configurations. Instances of these proteins have been given an different "chain code". As far as we are consered in our analysis we treat these as different proteins since they have a different structure. Hence our GOR algorithm should get the information of both configurations.

In [350]:
proteins = output.groupby(["PDB_code","PDB_chain_code"], sort=False)

Proteins with multpliple chain configurations:

In [371]:
dictionary = {}
for (proteinCode , chainCode), _ in proteins:
    if proteinCode in dictionary:
        print(proteinCode) # Just print such that we do not have to manually search later on in the dict.
        dictionary[proteinCode] += 1
    else:
        dictionary[proteinCode] = 1

1n7s
1wmh
1n7s
1n7s


We see that the protein instance "1n7s" comes in 4 configurations for the dssp data set and "1wmh" in 2. 

## Exploration of the skips:

In [373]:
print("Proteincode", "ChainCode","residuesSkipped")
for (proteinCode, chainCode), protein in proteins:
    rPosition= protein["PDB_seq_code"].values
    skipped = np.diff(rPosition)-1
    totalSkipped = np.sum(skipped)
    if totalSkipped!=0:
        print(proteinCode+" "+chainCode,totalSkipped) 

Proteincode ChainCode residuesSkipped
1j3a A 13
1mj4 A 1
1xod A 9
1jni A 5
2pvb A 1
2hly A 2
1ltz A 3
3r9f A 316
3lax A 3
1luc A 29
3mxz A 1
2pq7 A 43
1jl0 A 15
1nu0 A 7
3g02 A 7
2hc1 A 7
2iyv A 3
3bld A 24
1ekq A 15
2o9u X 1
1w5q A 10
1fye A 9
3ess A 9
1g4y B 1
3q62 A 2
3gp6 A 8
2ril A 4
3bs4 A 8
3b4q A 3
1dg6 A 13
1qg8 A 17
2oqz A 5
3elf A 13
1nqj A 4
2vpa A 11
3jud A 6
1nnx A 13
1b8z A 23
3bvf A 834
1od6 A 5
1oi0 A 13
2r4q A 2
1nn5 A 5
1vke A 1
2fco A 17
1vju A 11
1xbi A 1
1v05 A 2633
3lb2 A 4
1v2b A 23
2bog X 6
1zk5 A 3
2pag A 3
2prx A 32
1wpu A 1852
2icu A 10
1mnn A 13
3bt5 A 2
1m1f A 3
2eng A 5
2v9l A 1
2za4 B 1
1ymt A 8
1s9u A 6
2qt1 A 5
1m22 A 3
2okf A 7
1tzp A 15
2ij2 A 3
1usm A 3
3b5o A 7
2aeb A 232
2ap3 A 5
3fcx A 14
1cxq A 4
1ryl A 7
1k4n A 7
2gyq A 7
2qzc A 2
1xju A 2
2wnv B 1
2fup A 2
3h7c X 7
1fcq A 7
1z67 A 2
1jhg A 2
2brf A 1
2ggc A 836
1p5z B 12
1f9v A 29
1gpp A 239
1vyk A 20
3bhd A 3
1df4 A 5
3eoj A 2
2zfg A 1
1x9d A 1
2y1q A 5
3ed7 A 23
1z72 A 1
1lyq A 3
1ujp A 28
2

We see that actually many proteins have just missing entries. For instance the first one has directly apperentally 13 missing. 

Let's have a closer look to where exactly this happens:

In [375]:
def printSkips(code, chainCode):
    rPosition = proteins.get_group((code, chainCode))["PDB_seq_code"].values
    skipped = np.diff(rPosition)-1
    indexesSkipped = np.where(skipped!=0)[0]
    print("Skipped exactly here:")
    for index in indexesSkipped:
        print(f"[{rPosition[index]},{rPosition[index+1]}]")
    print("Given sequence positions:")
    print(rPosition)
printSkips(code = "1j3a",chainCode = "A" )

Skipped exactly here:
[53,67]
Given sequence position:
[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  67
  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85
  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101 102 103
 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121
 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139
 140 141 142]


We see that the skip happens at position 53 -> 67

In order to really understand the data and see indeed that it meant that indeed 13 entries were missing and that we could indeed trust the `PDB_seq_code` as the sequence number of the proteins. I had a look at the PDB for protein "1j3a", and indeed looking around in the files I found:

> REMARK 465                                                                      
REMARK 465 MISSING RESIDUES                                                     
REMARK 465 THE FOLLOWING RESIDUES WERE NOT LOCATED IN THE                       
REMARK 465 EXPERIMENT. (M=MODEL NUMBER; RES=RESIDUE NAME; C=CHAIN               
REMARK 465 IDENTIFIER; SSSEQ=SEQUENCE NUMBER; I=INSERTION CODE.)                
REMARK 465                                                                      
REMARK 465   M RES C SSSEQI                                                     
REMARK 465     GLY A    54                                                      
REMARK 465     LEU A    55                                                      
REMARK 465     ARG A    56                                                      
REMARK 465     THR A    57                                                      
REMARK 465     LEU A    58                                                      
REMARK 465     THR A    59                                                      
REMARK 465     ASN A    60                                                      
REMARK 465     PRO A    61                                                      
REMARK 465     ARG A    62                                                      
REMARK 465     ARG A    63                                                      
REMARK 465     GLY A    64                                                      
REMARK 465     PRO A    65                                                      
REMARK 465     PHE A    66 

So indeed the protein sequence is know for sequence number 54->66. But in the experiment they could not locate the position of the residues, hence one can then also not say in what structure they are organised. It's for this reason that those entries are simply not listed in our input data.

And hence we can assume going forward that this is the case with all other proteins that have missing entries.

---
So we found appart from residue entries that are not one of the standard 20 amino acids. We also have entries where the SS is simply not know. And these we could detect as presented above. 

Playing around with the function `printSkips` to investigate it is found that most often the proteins with extremely large skips >1000. That those from entries where just a few residues are added in the beginning or the ending. These are so distant from the main sequence that they do not contain any information from neighbouring residues. However, we will keep them since they still provide information on that particular residues itself like in GOR I.


We see that it is also sometimes possible that multiple skipps happen, as is seen in protein number:"1gpp"

### How to deal with the skips:

It is important that the relative distance between residues is respected, since this is essential in our GOR analysis! 

As was said with the residues, we will flag invalid residue entries with -1 to let the running window know to ignore these entries. What we will do with the skips is add new entries to our data that will also be flagged as invalid since for those added entries we do not know the SS. 

It is important that we add these, since if for instance a skip of 1 happens well then the amino acid on the left of the skip needs to know that the amino acid on the right is not its neighbour (m=1) but its next to neirest neighbour (m=2). 

We only add up to 8 invalid dummy entries. Since one side of our window is only 8 wide (eg. [-8 to -1] and [1 to 8]) and invalid entries are automatically skipped when the window at m=0 goes over them. Thus skips larger then 8 only get 8 invalid dummy entries added between them, this then also ensures that our data does not grow unnecassary big if a skip of 2000 happens.

## Further prepare the input data for analysis.

As said above we now need to add our dummy entrances.

We will not be doing this in the pandas dataframe since then we also need entries for the other columns while we just want to focus on the sequence. So we just create our own dictionary and store there the updated sequences in numpy format.

In [ ]:
def createDataset(dataSet = "dssp", protFamily = None):
    pd.read_csv("inputData/"+dataSet+"_info.txt", sep="\t", header=None, names=["PDB_code", "PDB_chain_code", "PDB_seq_code", "residue_name", "SS"])
    